In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os


c:\users\chris\anaconda3\envs\tf2019\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\chris\anaconda3\envs\tf2019\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\chris\anaconda3\envs\tf2019\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\chris\anaconda3\envs\tf2019\lib\site-packages\tensorflow\pyth

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model


Using TensorFlow backend.


In [6]:
df_train = pd.read_csv('./action-items-data-chris/train_balanced_4.csv')
df_dev = pd.read_csv('./action-items-data-chris/dev_balanced_4.csv')
df_test = pd.read_csv('./action-items-data-chris/test_balanced_labelled_4.csv')


In [7]:
glove_dir = 'C:/Users/chris/Documents/AI/nlp/glove'

In [8]:
print(len(df_train))
print(len(df_dev))
print(len(df_test))


1698
300
184


In [9]:
df_train_and_dev = pd.concat([df_train, df_dev]) #let's combine the utterances in the train and dev set


In [10]:
trainAndDevUtterances = df_train_and_dev['utterance'].tolist()
train_dev_labels = df_train_and_dev['is_action'].tolist()

trainLabels = df_train['is_action'].tolist()
devLabels = df_dev['is_action'].tolist()


In [11]:
testUtterances = df_test['utterance'].tolist()
test_labels = df_test['label'].tolist()


In [12]:
maxlen = 60
training_samples = len(df_train)
validation_samples = len(df_dev)
max_words = 10000


In [13]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(trainAndDevUtterances) #I could have included test utterances here too I think
sequences = tokenizer.texts_to_sequences(trainAndDevUtterances)

word_index = tokenizer.word_index
print("Found %s unique tokens" % len(word_index))


Found 3781 unique tokens


In [14]:
testsequences = tokenizer.texts_to_sequences(testUtterances)


In [15]:
data = pad_sequences(sequences,maxlen=maxlen)
labels = np.asarray(train_dev_labels)
print("Shape of data tensor:", data.shape)
print("Shape of label tensor:", labels.shape)


Shape of data tensor: (1998, 60)
Shape of label tensor: (1998,)


In [16]:
testdata = pad_sequences(testsequences,maxlen=maxlen)
testlabels = np.asarray(test_labels)
print("Shape of test data tensor:", testdata.shape)
print("Shape of test label tensor:", testlabels.shape)


Shape of test data tensor: (184, 60)
Shape of test label tensor: (184,)


In [17]:
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples:training_samples + validation_samples]
y_val = labels[training_samples:training_samples + validation_samples]



In [19]:
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embeddings_index[word] = coefs
f.close()

print("Found %s glove word vectors" % len(embeddings_index))


Found 400000 glove word vectors


In [20]:
embedding_dim = 100

#This is where the vocabulary we built with our tokenizer, in which words in our action items
#dataset are assigned unique index integers, get assigned their corresponding glove vectors.

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    

In [21]:
from keras.models import Sequential
from keras.layers import Embedding, Dense
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))


In [22]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False


In [23]:
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])
history = model.fit(x_train, y_train,
                   epochs=40,
                   batch_size=32,
                   validation_data=(x_val, y_val))

#model.save_weights('pre_trained_glove_model_weights.h5')



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1698 samples, validate on 300 samples
Epoch 1/40
1698/1698 [==============================] - 5s 3ms/step - loss: 0.6423 - acc: 0.6313 - val_loss: 0.5934 - val_acc: 0.6933
Epoch 2/40
1698/1698 [==============================] - 4s 2ms/step - loss: 0.5717 - acc: 0.7250 - val_loss: 0.6648 - val_acc: 0.6167
Epoch 3/40
1698/1698 [==============================] - 4s 2ms/step - loss: 0.5390 - acc: 0.7344 - val_loss: 1.0152 - val_acc: 0.5333
Epoch 4/40
1698/1698 [==============================] - 4s 2ms/step - loss: 0.5275 - acc: 0.7473 - val_loss: 0.7044 - val_acc: 0.6167
Epoch 5/40
1698/1698 [==============================] - 4s 2ms/step - loss: 0.5042 - acc: 0.7650 - val_loss: 0.7012 - val_acc: 0.6767
Epoch 6/40
1698/1698 [==============================] - 4s 2ms/step - loss: 0.4717 - acc: 0.7903 - val_loss: 0.8008 - val_acc: 0.6000
Epoch 7/40
1698/1698 [==============================] 

In [24]:
model.save('./rnn-glove-action-item-classifier.h5')  # creates a HDF5 file 
#This saves and allows loading of whole model: architecture + weights + optimizer state
#see https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model to find out
#how to save just a model's weights


In [25]:
loaded_model = load_model('./rnn-glove-action-item-classifier.h5')


In [26]:
loaded_model.evaluate(testdata, testlabels)


184/184 [==============================] - 0s 1ms/step


[0.9743937927743663, 0.7771739104519719]

In [27]:
print(loaded_model.metrics_names)


['loss', 'acc']


In [28]:
test_predictions = loaded_model.predict(testdata)
print(test_predictions[0:10])


[[0.99999905]
 [0.97770834]
 [0.99972343]
 [0.07447377]
 [0.00279701]
 [0.5196131 ]
 [0.4372197 ]
 [0.99999547]
 [0.01376092]
 [0.1564514 ]]


In [29]:
test_binary_predictions = test_predictions >= 0.5
test_binary_predictions = test_binary_predictions.astype(int)
test_binary_predictions = test_binary_predictions.flatten()
print(test_binary_predictions)


[1 1 1 0 0 1 0 1 0 0 1 0 1 1 0 0 0 1 1 0 1 1 1 1 1 0 1 0 1 0 0 1 1 0 1 1 0
 1 0 1 1 1 1 1 0 0 1 0 1 1 1 1 0 0 1 0 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0
 0 1 1 1 0 1 0 1 1 1 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 0 1 1 0 0 1 0
 0 1 1 0 0 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0
 1 1 1 0 1 0 1 1 1 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 1 0 1 1]


In [30]:
print(testlabels)


[1 1 1 0 0 1 0 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 1 1 1 0 1 0 1 0 0 1 1 1 1 1 0
 1 1 0 1 1 1 1 0 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 1 0 1 1 1 1 0
 0 0 1 1 0 1 0 1 1 1 0 1 0 0 1 1 0 1 0 1 1 0 0 0 0 0 1 1 1 1 0 1 1 1 0 1 1
 1 1 0 0 0 1 0 1 0 1 0 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 1 1 1 1 1 0 0 1 1 1 0
 1 0 1 1 1 1 1 0 0 1 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 1 1 0 0 1 1 1 1]


In [31]:
def evaluateResults(classifierLabels, humanLabels):
    
    numPredictions = len(classifierLabels)
    truePositives = 0
    trueNegatives = 0

    falsePositives = 0
    falseNegatives = 0
    numCorrect = 0
    numWrong = 0

    for i in range(len(humanLabels)): # assuming the lists are of the same length
        if (humanLabels[i]==1):
            if (classifierLabels[i] == 1):
                truePositives = truePositives + 1
                numCorrect = numCorrect + 1
            elif(classifierLabels[i] == 0):
                falseNegatives = falseNegatives + 1
                numWrong = numWrong + 1
        elif(humanLabels[i]==0):
            if (classifierLabels[i] == 0):
                trueNegatives = trueNegatives + 1
                numCorrect = numCorrect + 1
            elif(classifierLabels[i]==1):
                falsePositives = falsePositives + 1
                numWrong = numWrong + 1


    print("true positives:", truePositives)
    print("false negatives:", falseNegatives)
    print("false positives:", falsePositives)
    print()

    accuracy = numCorrect/numPredictions
    precision = truePositives/(truePositives + falsePositives)
    recall = truePositives/(truePositives + falseNegatives)
    
    return (accuracy,precision,recall)


In [32]:
acc,prec,recall = evaluateResults(test_binary_predictions, testlabels)
print("accuracy:", acc)
print("precision:", prec)
print("recall:", recall)


true positives: 90
false negatives: 18
false positives: 23

accuracy: 0.7771739130434783
precision: 0.7964601769911505
recall: 0.8333333333333334


In [33]:
f1 = (2 * prec * recall)/(prec + recall)
print(f1)


0.8144796380090498
